This notebook demonstrates the creation and usage of a chatbot powered by a generative AI model. Below is an overview of the workflow and the components involved:

## 1. Environment Setup
The notebook begins by importing necessary modules and loading environment variables using `dotenv`. This ensures that API keys and other sensitive information are securely loaded.

## 2. Chatbot Model Initialization
The generative AI model (`google_genai:gemini-2.5-pro`) is initialized using the `init_chat_model` function. This model is used to generate responses based on user input.

## 3. State Management
A custom `State` type is defined using `TypedDict`. This state keeps track of the conversation messages exchanged between the user and the chatbot.

## 4. State Graph Construction
A `StateGraph` is built to define the flow of the chatbot. The graph includes:
- Nodes: Representing the chatbot logic.
- Edges: Connecting the start (`START`) and end (`END`) states with the chatbot node.

The graph is compiled into a `CompiledStateGraph` object, which can be invoked to process user input and generate responses.

## 5. Visualization
The structure of the state graph is visualized using `IPython.display` and rendered as an image. This helps in understanding the flow of the chatbot's logic.

## 6. Chatbot Interaction
The chatbot is tested with a sample user message (`"What is the capital of France?"`). The graph processes the input and generates a response (`"The capital of France is **Paris**."`).

## 7. Interactive Chatbot (Optional)
An interactive chatbot loop is implemented (commented out) to allow continuous conversation with the user. The loop processes user input, updates the state, and generates responses until the user exits.

## Key Variables
- **START** and **END**: Represent the start and end states of the graph.
- **State**: Defines the structure of the chatbot's state.
- **builder**: Used to construct the state graph.
- **graph**: The compiled state graph for invoking the chatbot.
- **messages**: Stores the user input and chatbot responses.
- **response**: Contains the chatbot's reply to the user's query.
- **state**: Maintains the conversation history during interactive sessions.

This notebook provides a structured approach to building and interacting with a generative AI-powered chatbot using state graphs and visualization tools.

In [ ]:
from dotenv import load_dotenv
from typing_extensions import TypedDict
from typing import Annotated
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

load_dotenv()

In [ ]:
llm = init_chat_model("google_genai:gemini-2.5-pro")

class State(TypedDict):
    messages: Annotated[list, add_messages]
    
def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages = {"role": "user", "content": "What is the capital of France?"}
    

response = graph.invoke({"messages": [messages]})

# response["messages"]
print("Bot:", response["messages"][1].content)

In [ ]:
# state = None

# while True:
#     input_msg = input("You: ")
#     if input_msg.lower() in ["exit", "quit"]:
#         print("Exiting chatbot.")
#         break
#     if state is None:
#         state: State = {"messages": [{"role": "user", "content": input_msg}]}
#     else:
#         state["messages"].append({"role": "user", "content": input_msg})
        
#     state = graph.invoke(state)
#     print("Bot:", state["messages"][-1].content)